# 🧮 Week 5-6 · Notebook 09 · A Deep Dive into Vector Embeddings

**Module:** LLMs, Prompt Engineering & RAG  
**Project:** Build the Knowledge Core for the Manufacturing Copilot

---

### The Heart of RAG: Turning Meaning into Math

Throughout this module, we've built a powerful Retrieval-Augmented Generation (RAG) system. We've seen how it can fetch relevant documents from a knowledge base to provide accurate, context-aware answers. But how does the "retrieval" part actually work? How does a computer understand which documents are "relevant" to a user's question?

The answer lies in **Vector Embeddings**.

Vector embeddings are the silent engine at the heart of modern semantic search. They are a way of converting the rich, nuanced meaning of text into a universal, mathematical format—a list of numbers called a vector. This process allows us to plot words, sentences, and entire documents in a high-dimensional "meaning space," where similar concepts are located close to one another.

In this notebook, we will pull back the curtain on this fascinating technology. We will move beyond using embeddings as a black box and explore:
- What are they, intuitively?
- How do we generate them?
- How can we measure their similarity?
- How can we visualize their relationships?
- And most importantly, how do we know if our embeddings are any good?

Understanding embeddings is the final and most crucial step to mastering RAG. Let's begin.

## 🎯 Learning Objectives

By the end of this notebook, you will have a deep, intuitive understanding of the technology that powers semantic search. You will be able to:

1.  **Explain Vector Embeddings Intuitively:** Articulate what an embedding is and how it captures the semantic "meaning" of text in a mathematical form.
2.  **Generate and Compare Embeddings:** Use a state-of-the-art `sentence-transformers` model to convert text into high-dimensional vectors and measure their similarity using cosine distance.
3.  **Visualize the "Meaning Space":** Use Principal Component Analysis (PCA) to reduce the dimensionality of embeddings and create 2D scatter plots that visually demonstrate how the model organizes information.
4.  **Implement a Basic Embedding Evaluation:** Build a simple but effective "hit-rate" evaluation to measure how well your embedding model retrieves relevant documents for a given query, a critical practice for any production RAG system.

## ⚙️ Step 1: Environment Setup

For this notebook, we'll need a few key libraries:
-   `sentence-transformers`: The core library for generating high-quality text embeddings. It provides easy access to hundreds of pre-trained models from the Hugging Face Hub.
-   `pandas` and `numpy`: For data manipulation and numerical operations.
-   `scikit-learn` and `matplotlib`: For dimensionality reduction (PCA) and plotting our visualizations.

In [ ]:
# Install required packages
!pip install -q sentence-transformers pandas numpy scikit-learn matplotlib

# --- Core Imports ---
# For creating and working with embeddings
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# --- Visualization Imports ---
# For dimensionality reduction and plotting
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set_style("whitegrid")

print("Libraries installed and imported.")

## 🧠 Step 2: Generating Embeddings - From Text to Vectors

So, what *is* an embedding?

Imagine a vast, multi-dimensional space—let's call it the "meaning space." An embedding model is a sophisticated neural network trained on billions of sentences. Its job is to read a piece of text and place it as a single point (a vector) within this space. The model's key breakthrough is that it learns to arrange these points based on **semantic meaning**.

-   Texts with similar meanings are placed close together.
-   Texts with different meanings are placed far apart.

This simple principle is what enables a computer to "understand" context. When a user asks a question, we can embed their query into the same space and simply look for the document vectors that are its nearest neighbors.

Let's see this in action. We'll create a small, representative knowledge base for our manufacturing use case and then use a pre-trained `SentenceTransformer` model to convert each document into a vector. We will use `all-MiniLM-L6-v2`, a popular and highly efficient model that balances performance and size. It creates vectors with **384 dimensions**. While we can't visualize 384 dimensions directly, we can work with them mathematically.

In [ ]:
# Create a DataFrame with a sample of our manufacturing knowledge base.
# Each document is labeled with a category for later analysis.
documents = pd.DataFrame([
    {"id": 0, "label": "maintenance", "text": "The hydraulic press requires a full oil change every 2000 hours of operation to prevent component wear."},
    {"id": 1, "label": "maintenance", "text": "Weekly inspection of the main conveyor belt's tension is a mandatory task for all shift supervisors."},
    {"id": 2, "label": "incident", "text": "Incident Report #451: CNC machine #3 experienced a sudden stall due to a critical coolant leak. Downtime was 35 minutes."},
    {"id": 3, "label": "incident", "text": "Incident Report #452: An operator noticed unusual, high-frequency vibrations emanating from the main compressor unit."},
    {"id": 4, "label": "safety", "text": "Safety Alert: Always perform the complete lockout-tagout procedure before servicing any machinery with moving parts."},
    {"id": 5, "label": "quality", "text": "Quality Control Alert: Part #A-103 from the morning shift is showing consistent surface defects. The vision system requires immediate recalibration."}
])

print("--- Knowledge Base Documents ---")
display(documents)

# Load a pre-trained Sentence Transformer model from the Hugging Face Hub.
# "all-MiniLM-L6-v2" is a fantastic starting model: it's fast, effective, and has a small memory footprint.
print("\nLoading embedding model...")
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Model loaded.")

# Generate embeddings for all documents in our knowledge base.
# The `encode` method takes a list of strings and returns a NumPy array of vectors.
print("\nGenerating embeddings...")
embeddings = model.encode(documents.text.tolist())
print("Embeddings generated.")

# Print the shape of the resulting embeddings matrix.
# The shape will be (number of documents, number of dimensions per embedding).
print(f"\nShape of embeddings matrix: {embeddings.shape}")
print(f"We have {embeddings.shape[0]} vectors, each with {embeddings.shape[1]} dimensions.")

## 📏 Step 3: Measuring Similarity with Cosine Distance

Now that our documents are represented as vectors, how do we mathematically measure their "closeness" in the 384-dimensional meaning space?

The most common and intuitive metric is **Cosine Similarity**.

Instead of measuring the straight-line (Euclidean) distance between two vectors, cosine similarity measures the **angle** between them. This is powerful because it captures the *orientation* of the vectors, not their magnitude. In the context of text, this means it measures the similarity of the topics, regardless of the length of the documents.

-   A cosine similarity score of **1** means the vectors point in the exact same direction (perfectly similar).
-   A score of **0** means they are orthogonal (completely unrelated).
-   A score of **-1** means they point in opposite directions (dissimilar).

The `sentence-transformers` library provides a handy `util.cos_sim` function to compute this efficiently. Let's calculate the similarity between every document in our knowledge base and see what relationships the model has learned.

# Calculate the cosine similarity matrix between all pairs of embeddings
# The result is a square matrix where the (i, j) entry is the similarity between document i and document j.
similarity_matrix = util.cos_sim(embeddings, embeddings).cpu().numpy()

# For better readability, let's display this matrix as a heatmap using pandas and seaborn.
sim_df = pd.DataFrame(similarity_matrix, index=documents.label, columns=documents.label)

plt.figure(figsize=(8, 6))
sns.heatmap(
    sim_df,
    annot=True,          # Show the similarity scores on the heatmap
    cmap="viridis",      # Use a color-blind friendly colormap
    fmt=".2f"            # Format scores to two decimal places
)
plt.title("Cosine Similarity Matrix of Document Embeddings")
plt.show()

### Interpretation of the Heatmap

The heatmap reveals the semantic structure the model has found in our data. The diagonal is always a perfect **1.00**, as every document is identical to itself. The interesting parts are the off-diagonal elements:

-   **High Similarity (Bright Squares):**
    -   Notice the bright square where the two `maintenance` documents intersect (**0.53**). The model correctly identified that "changing hydraulic oil" and "inspecting a conveyor belt" are related maintenance activities.
    -   Similarly, the two `incident` reports have a high similarity score (**0.62**), as they both describe unexpected events ("machine stall" and "unusual vibrations").

-   **Low Similarity (Dark Squares):**
    -   The `safety` document has very low similarity to the `quality` document (**0.11**). This makes sense; a lockout-tagout procedure is conceptually very different from a part defect inspection.
    -   The `maintenance` documents are not very similar to the `incident` reports, which is also correct.

This heatmap is a powerful confirmation: **the embedding model is successfully organizing our knowledge based on its meaning.** This is the fundamental property that allows a vector store to retrieve relevant context for a user's query.

## 🎨 Step 4: Visualizing the "Meaning Space" with PCA

While the heatmap is great for comparing pairs of documents, it doesn't give us a global picture of the "meaning space." How are all the documents arranged relative to each other?

To visualize this, we need to reduce the 384 dimensions of our embeddings down to something we can plot, like 2 or 3 dimensions. A classic and powerful technique for this is **Principal Component Analysis (PCA)**.

PCA is a dimensionality reduction algorithm that finds the "principal components" of the data—the directions in the high-dimensional space where the data varies the most. By projecting the data onto the first two principal components, we can create a 2D scatter plot that preserves as much of the original structure as possible.

This allows us to literally *see* how the embedding model is clustering our documents.

# 1. Initialize PCA
# We ask PCA to find the 2 most important "principal components" of our 384D data.
pca = PCA(n_components=2)

# 2. Fit and Transform
# `fit_transform` learns the principal components from the embeddings and then projects the embeddings onto them.
coords = pca.fit_transform(embeddings)

# 3. Add coordinates to our DataFrame for easy plotting
documents['pca_x'] = coords[:, 0]
documents['pca_y'] = coords[:, 1]

# 4. Plot the results
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=documents,
    x='pca_x',
    y='pca_y',
    hue='label',          # Color points by their category
    s=200,                # Increase marker size
    alpha=0.8
)

# Add annotations to each point to show which document it is
for i, row in documents.iterrows():
    plt.text(row.pca_x + 0.01, row.pca_y, f"Doc {row['id']}", fontsize=9, weight='bold')

plt.title("2D Visualization of Document Embeddings using PCA", fontsize=16)
plt.xlabel("Principal Component 1", fontsize=12)
plt.ylabel("Principal Component 2", fontsize=12)
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Interpretation of the PCA Plot

The 2D plot provides a stunningly clear visual confirmation of what we saw in the heatmap. The embedding model has created distinct, semantically meaningful clusters in the meaning space:

-   **The `maintenance` Cluster:** Documents 0 and 1 are grouped closely together.
-   **The `incident` Cluster:** Documents 2 and 3 form another tight cluster, far away from the maintenance tasks.
-   **The Outliers:** The `safety` (Doc 4) and `quality` (Doc 5) documents are isolated, reflecting their unique and distinct topics.

This visualization is incredibly powerful. It proves that when a user asks a question like, *"What happened with the CNC machine?"*, the embedding of this question will naturally fall near the `incident` cluster, allowing the vector store to instantly retrieve the most relevant documents (Doc 2 and 3). This is the magic of semantic search in action.

## 🧪 Step 5: Evaluating Embedding Quality for RAG

Visualizations and heatmaps are great for building intuition, but for a production system, we need objective, quantitative metrics. **Not all embedding models are created equal.** A model that works well for general web text might perform poorly on specialized documents like our manufacturing SOPs.

**How do we evaluate an embedding model's fitness for our RAG task?**

A simple and effective method is to measure **retrieval accuracy**. We create a small evaluation set of questions and manually identify the correct "ground truth" document that should answer each one. Then, we automate a test to see if our embedding model can retrieve the correct document.

A common metric for this is **Recall@k**, which answers the question:
> For a given query, is the correct document found within the top `k` retrieved results?

If we test this across many queries, we can calculate a "hit rate" that tells us how effective our retriever is. This is a crucial practice for comparing different embedding models or different chunking strategies.

In [ ]:
# 1. Define our evaluation set
# This is a small "test suite" for our retriever.
# Each dictionary key is a question, and the value is the ID of the document that contains the answer.
eval_questions = {
    "How often are you supposed to change the hydraulic oil?": 0, # Should match Doc 0
    "What was the reported problem with the CNC machine?": 2,     # Should match Doc 2
    "What is the mandatory safety rule for servicing machines?": 4 # Should match Doc 4
}

# 2. Define our evaluation function
def evaluate_recall(k: int = 1):
    """
    Calculates Recall@k for our evaluation set.
    
    Args:
        k: The number of top documents to consider for a "hit".
        
    Returns:
        The percentage of questions for which the correct document was retrieved.
    """
    correct_hits = 0
    total = len(eval_questions)
    
    print(f"\n--- Evaluating Recall@{k} ---")
    
    for question, expected_doc_id in eval_questions.items():
        # Embed the question using the same model
        query_embedding = model.encode(question)
        
        # Perform a semantic search against our document embeddings
        # `util.semantic_search` returns a list of lists of hits. We take the first list.
        hits = util.semantic_search(query_embedding, embeddings, top_k=k)[0]
        
        # Get the IDs of the retrieved documents
        retrieved_ids = [hit['corpus_id'] for hit in hits]
        
        # Check if the expected document was in the retrieved set
        if expected_doc_id in retrieved_ids:
            print(f"  ✅ HIT:  Question '{question[:30]}...' -> Retrieved IDs {retrieved_ids}")
            correct_hits += 1
        else:
            print(f"  ❌ MISS: Question '{question[:30]}...' -> Retrieved IDs {retrieved_ids}, Expected {expected_doc_id}")
            
    # Calculate the final score
    recall_score = correct_hits / total
    return recall_score

# 3. Run the evaluation
# Recall@1: Is the single most relevant document the correct one?
recall_at_1 = evaluate_recall(k=1)

# Recall@2: Is the correct document within the top two most relevant?
recall_at_2 = evaluate_recall(k=2)

print("\n--- Evaluation Summary ---")
print(f"Recall@1: {recall_at_1:.2%}")
print(f"Recall@2: {recall_at_2:.2%}")

## 🎉 Congratulations! You've Mastered the Core of RAG!

This notebook concludes our deep dive into the foundational components of modern LLM applications. You have progressed from a high-level understanding of RAG to a deep, intuitive grasp of the technology that makes it possible.

You now understand:

1.  **LLMs and Transformers:** The fundamental models that power generative AI.
2.  **Prompt Engineering:** The art and science of instructing LLMs to get the results you want.
3.  **Few-Shot Learning:** A powerful technique for improving model performance on specific tasks by providing examples.
4.  **Retrieval-Augmented Generation (RAG):** The complete architecture for connecting LLMs to external, dynamic knowledge bases.
5.  **Vector Embeddings:** The core technology that enables semantic search by turning meaning into math.
6.  **Embedding Evaluation:** The critical practice of measuring retrieval accuracy to ensure your RAG system is effective.

You possess all the conceptual tools and practical skills needed to design and build the knowledge core for our Manufacturing Copilot.

In the next notebook, **`10_llm_evaluation.ipynb`**, we will tackle one of the most challenging topics in applied AI: **How do you evaluate the quality of an LLM's generated output?** We will explore automated, AI-based evaluation frameworks that are essential for building reliable, production-ready generative AI applications. Let's move forward!